In [3]:
!pip install -q langchain langchain-community langchain-text-splitters
!pip install -q pypdf sentence-transformers faiss-cpu
!pip install -q transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.4 MB/s eta 0:00:00


In [4]:
# ============================================================================
# SECTION 2: IMPORT LIBRARIES
# ============================================================================

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import re
import json
from datetime import datetime

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [6]:
# ============================================================================
# SECTION 3: ENGINEERING DECISION TREES (BNBC 2020 DATA)
# ============================================================================

# Seismic Zone Data (Table 6.2.15 from BNBC 2020)
SEISMIC_ZONES = {
    'Dhaka': {'zone': 2, 'Z': 0.20, 'intensity': 'Moderate'},
    'Sylhet': {'zone': 4, 'Z': 0.36, 'intensity': 'Very Severe'},
    'Chittagong': {'zone': 3, 'Z': 0.28, 'intensity': 'Severe'},
    'Rajshahi': {'zone': 1, 'Z': 0.12, 'intensity': 'Low'},
    'Khulna': {'zone': 1, 'Z': 0.12, 'intensity': 'Low'},
    'Barisal': {'zone': 1, 'Z': 0.12, 'intensity': 'Low'},
    'Rangpur': {'zone': 3, 'Z': 0.28, 'intensity': 'Severe'},
    'Mymensingh': {'zone': 4, 'Z': 0.36, 'intensity': 'Very Severe'},
    'Comilla': {'zone': 2, 'Z': 0.20, 'intensity': 'Moderate'},
    'Brahmanbaria': {'zone': 3, 'Z': 0.28, 'intensity': 'Severe'},
    'Gazipur': {'zone': 2, 'Z': 0.20, 'intensity': 'Moderate'},
    'Narayanganj': {'zone': 2, 'Z': 0.20, 'intensity': 'Moderate'},
}

# Soil Classification (Table 6.2.13 from BNBC 2020)
SOIL_CLASSES = {
    'rock': {
        'class': 'SA',
        'S': 1.0,
        'TB': 0.15,
        'TC': 0.40,
        'TD': 2.0,
        'description': 'Rock/Hard soil',
        'Vs': '> 800 m/s'
    },
    'very_dense_sand': {
        'class': 'SB',
        'S': 1.2,
        'TB': 0.15,
        'TC': 0.50,
        'TD': 2.0,
        'description': 'Very dense sand/stiff clay',
        'Vs': '360-800 m/s'
    },
    'dense_sand': {
        'class': 'SC',
        'S': 1.15,
        'TB': 0.20,
        'TC': 0.60,
        'TD': 2.0,
        'description': 'Dense/Medium sand',
        'Vs': '180-360 m/s'
    },
    'loose_sand': {
        'class': 'SD',
        'S': 1.35,
        'TB': 0.20,
        'TC': 0.80,
        'TD': 2.0,
        'description': 'Loose cohesionless soil',
        'Vs': '< 180 m/s'
    },
    'soft_clay': {
        'class': 'SE',
        'S': 1.4,
        'TB': 0.15,
        'TC': 0.50,
        'TD': 2.0,
        'description': 'Soft alluvium layer',
        'Vs': 'Variable'
    },
    'marshland': {
        'class': 'SD',
        'S': 1.35,
        'TB': 0.20,
        'TC': 0.80,
        'TD': 2.0,
        'description': 'Soft cohesive soil',
        'Vs': '< 180 m/s'
    },
}

# Building Importance Factor (Table 6.2.17 from BNBC 2020)
BUILDING_IMPORTANCE = {
    'residential': {'category': 'II', 'I': 1.00, 'description': 'Normal residential'},
    'commercial': {'category': 'II', 'I': 1.00, 'description': 'Commercial buildings'},
    'apartment': {'category': 'II', 'I': 1.00, 'description': 'Apartment buildings'},
    'school': {'category': 'III', 'I': 1.25, 'description': 'Educational facilities'},
    'hospital': {'category': 'IV', 'I': 1.50, 'description': 'Essential facilities'},
    'emergency_shelter': {'category': 'IV', 'I': 1.50, 'description': 'Emergency facilities'},
    'fire_station': {'category': 'IV', 'I': 1.50, 'description': 'Fire and rescue'},
    'power_plant': {'category': 'IV', 'I': 1.50, 'description': 'Critical utilities'},
}

# Structural Systems (Table 6.2.19 from BNBC 2020)
STRUCTURAL_SYSTEMS = {
    'special_rc_shear_wall': {
        'R': 5,
        'Omega': 2.5,
        'Cd': 5,
        'height_limit_B': 'NL',
        'height_limit_C': 'NL',
        'height_limit_D': 50,
        'description': 'Special RC shear walls'
    },
    'ordinary_rc_shear_wall': {
        'R': 4,
        'Omega': 2.5,
        'Cd': 4,
        'height_limit_B': 'NL',
        'height_limit_C': 'NL',
        'height_limit_D': 'NP',
        'description': 'Ordinary RC shear walls'
    },
    'special_rc_moment_frame': {
        'R': 8,
        'Omega': 3,
        'Cd': 5.5,
        'height_limit_B': 'NL',
        'height_limit_C': 'NL',
        'height_limit_D': 'NL',
        'description': 'Special RC moment frames'
    },
    'ordinary_rc_moment_frame': {
        'R': 3,
        'Omega': 3,
        'Cd': 2.5,
        'height_limit_B': 'NL',
        'height_limit_C': 'NL',
        'height_limit_D': 'NP',
        'description': 'Ordinary RC moment frames'
    },
    'dual_system_special': {
        'R': 7,
        'Omega': 2.5,
        'Cd': 5.5,
        'height_limit_B': 'NL',
        'height_limit_C': 'NL',
        'height_limit_D': 'NL',
        'description': 'Dual system with special frames'
    },
}

print("✅ Engineering decision trees loaded!")
print(f"   - {len(SEISMIC_ZONES)} seismic zones")
print(f"   - {len(SOIL_CLASSES)} soil classifications")
print(f"   - {len(BUILDING_IMPORTANCE)} building types")
print(f"   - {len(STRUCTURAL_SYSTEMS)} structural systems")

✅ Engineering decision trees loaded!
   - 12 seismic zones
   - 6 soil classifications
   - 8 building types
   - 5 structural systems


In [12]:
# ============================================================================
# SECTION 4: LOAD AND PROCESS BNBC 2020 PDF
# ============================================================================

# PDF file path
PDF_PATH = "/BNBC 2020_Earthquake load.pdf"

print("=" * 80)
print("/BNBC 2020_Earthquake load.pdf")
print("=" * 80)

# Step 1: Load PDF
print("\n[1/3] Loading PDF file...")
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
print(f"      ✓ Loaded {len(documents)} pages")

# Step 2: Split into chunks
print("\n[2/3] Splitting into semantic chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
texts = text_splitter.split_documents(documents)
print(f"      ✓ Created {len(texts)} text chunks")

# Step 3: Show sample chunk
print("\n[3/3] Sample chunk:")
print("-" * 80)
print(texts[0].page_content[:300] + "...")
print("-" * 80)

print(f"\n✅ PDF processing complete!")

/BNBC 2020_Earthquake load.pdf

[1/3] Loading PDF file...
      ✓ Loaded 64 pages

[2/3] Splitting into semantic chunks...
      ✓ Created 172 text chunks

[3/3] Sample chunk:
--------------------------------------------------------------------------------
evsjv‡`k †M‡RU, AwZwi³, †deªæqvwi 11, 2021 3185    
 
  
 
2.5  Earthquake Loads  
2.5.1   General 
Minimum design earthquake forces for buildings, structures or components 
thereof shall be determined in accordance with the provisions of Sec 2.5. Some 
definitions and symbols relevant for earthquak...
--------------------------------------------------------------------------------

✅ PDF processing complete!


In [13]:
# ============================================================================
# SECTION 5: CREATE VECTOR DATABASE (RAG CORE)
# ============================================================================

print("=" * 80)
print("CREATING VECTOR DATABASE")
print("=" * 80)

# Step 1: Initialize embeddings model
print("\n[1/2] Initializing embeddings model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("      ✓ Embeddings model ready")

# Step 2: Build FAISS vector database
print("\n[2/2] Building FAISS vector database...")
vectorstore = FAISS.from_documents(texts, embeddings)
print("      ✓ Vector database created")

print(f"\n✅ RAG system ready with {len(texts)} indexed chunks!")

CREATING VECTOR DATABASE

[1/2] Initializing embeddings model...


/tmp/ipython-input-3032269902.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

      ✓ Embeddings model ready

[2/2] Building FAISS vector database...
      ✓ Vector database created

✅ RAG system ready with 172 indexed chunks!


In [14]:
# ============================================================================
# SECTION 6: PARAMETER EXTRACTION FUNCTIONS
# ============================================================================

def extract_location(query):
    """Extract location from query"""
    for city in SEISMIC_ZONES.keys():
        if city.lower() in query.lower():
            return city
    return None

def extract_soil_type(query):
    """Extract soil type from query"""
    soil_keywords = {
        'rock': ['rock', 'hard soil', 'bedrock'],
        'very_dense_sand': ['very dense', 'dense sand', 'stiff', 'very stiff'],
        'dense_sand': ['dense', 'medium dense', 'medium sand'],
        'loose_sand': ['loose', 'soft sand', 'loose soil'],
        'soft_clay': ['soft clay', 'alluvium', 'soft soil'],
        'marshland': ['marsh', 'swamp', 'wetland', 'marshland']
    }

    for soil, keywords in soil_keywords.items():
        if any(kw in query.lower() for kw in keywords):
            return soil
    return None

def extract_building_type(query):
    """Extract building type from query"""
    building_keywords = {
        'residential': ['residential', 'house', 'home', 'residence'],
        'commercial': ['commercial', 'office', 'shop', 'retail'],
        'apartment': ['apartment', 'flat', 'condo'],
        'school': ['school', 'university', 'college', 'educational'],
        'hospital': ['hospital', 'clinic', 'medical', 'healthcare'],
        'emergency_shelter': ['shelter', 'emergency', 'cyclone shelter'],
        'fire_station': ['fire station', 'fire department'],
        'power_plant': ['power plant', 'power station', 'utility']
    }

    for btype, keywords in building_keywords.items():
        if any(kw in query.lower() for kw in keywords):
            return btype
    return None

def extract_construction_year(query):
    """Extract construction year from query"""
    year_match = re.search(r'\b(19\d{2}|20\d{2})\b', query)
    if year_match:
        return int(year_match.group(1))
    return None

def extract_height(query):
    """Extract building height from query"""
    # Check for meters
    height_match = re.search(r'(\d+)\s*(m|meter|metre)', query.lower())
    if height_match:
        return int(height_match.group(1))

    # Check for stories/floors
    storey_match = re.search(r'(\d+)\s*(storey|story|floor)', query.lower())
    if storey_match:
        stories = int(storey_match.group(1))
        return stories * 3  # Assume 3m per floor

    return None

def extract_all_parameters(query):
    """Extract all engineering parameters from query"""
    params = {
        'location': extract_location(query),
        'soil_type': extract_soil_type(query),
        'building_type': extract_building_type(query),
        'construction_year': extract_construction_year(query),
        'height': extract_height(query),
    }
    return params

# Test the extraction
test_query = "I have a residential building in Dhaka built in 1995 on marshland soil"
test_params = extract_all_parameters(test_query)

print("✅ Parameter extraction functions ready!")
print("\nTest extraction:")
print(json.dumps(test_params, indent=2))

✅ Parameter extraction functions ready!

Test extraction:
{
  "location": "Dhaka",
  "soil_type": "marshland",
  "building_type": "residential",
  "construction_year": 1995,
  "height": null
}


In [15]:
# ============================================================================
# SECTION 7: RISK ASSESSMENT LOGIC
# ============================================================================

def calculate_seismic_risk(params):
    """
    Calculate seismic risk based on BNBC logic
    Risk score: 0-100
    - Zone factor: 0-40 points
    - Soil factor: 0-30 points
    - Construction year: 0-30 points
    """
    risk_score = 0
    risk_factors = []

    # 1. Seismic Zone Factor (0-40 points)
    if params['location']:
        zone_data = SEISMIC_ZONES.get(params['location'])
        if zone_data:
            zone = zone_data['zone']
            Z = zone_data['Z']

            if zone == 4:  # Very Severe
                risk_score += 40
                risk_factors.append(f"Very Severe seismic zone (Zone 4, Z={Z})")
            elif zone == 3:  # Severe
                risk_score += 30
                risk_factors.append(f"Severe seismic zone (Zone 3, Z={Z})")
            elif zone == 2:  # Moderate
                risk_score += 20
                risk_factors.append(f"Moderate seismic zone (Zone 2, Z={Z})")
            else:  # Low
                risk_score += 10
                risk_factors.append(f"Low seismic zone (Zone 1, Z={Z})")

    # 2. Soil Factor (0-30 points)
    if params['soil_type']:
        soil_data = SOIL_CLASSES.get(params['soil_type'])
        if soil_data:
            soil_class = soil_data['class']
            S = soil_data['S']

            if soil_class in ['SD', 'SE']:  # Poor soil
                risk_score += 30
                risk_factors.append(f"Poor soil condition (Class {soil_class}, S={S})")
            elif soil_class == 'SC':  # Medium soil
                risk_score += 20
                risk_factors.append(f"Medium soil condition (Class {soil_class}, S={S})")
            elif soil_class == 'SB':  # Good soil
                risk_score += 10
                risk_factors.append(f"Good soil condition (Class {soil_class}, S={S})")
            else:  # Rock
                risk_score += 5
                risk_factors.append(f"Excellent soil condition (Class {soil_class}, S={S})")

    # 3. Construction Year Factor (0-30 points)
    if params['construction_year']:
        year = params['construction_year']

        if year < 1993:  # Before BNBC 1993
            risk_score += 30
            risk_factors.append(f"Very old construction (built {year}, before BNBC 1993)")
        elif year < 2000:  # 1993-2000
            risk_score += 25
            risk_factors.append(f"Old construction (built {year}, before 2000)")
        elif year < 2010:  # 2000-2010
            risk_score += 18
            risk_factors.append(f"Moderate age construction (built {year})")
        elif year < 2020:  # 2010-2020
            risk_score += 10
            risk_factors.append(f"Recent construction (built {year})")
        else:  # After 2020
            risk_score += 5
            risk_factors.append(f"New construction (built {year}, post-BNBC 2020)")

    # Determine Risk Level
    if risk_score >= 70:
        risk_level = "CRITICAL"
        color = "🔴"
    elif risk_score >= 50:
        risk_level = "HIGH"
        color = "🟠"
    elif risk_score >= 30:
        risk_level = "MEDIUM"
        color = "🟡"
    else:
        risk_level = "LOW"
        color = "🟢"

    return {
        'risk_level': risk_level,
        'risk_score': risk_score,
        'risk_factors': risk_factors,
        'color': color
    }

# Test risk calculation
test_params = {
    'location': 'Dhaka',
    'soil_type': 'marshland',
    'construction_year': 1995,
    'building_type': 'residential',
    'height': None
}

test_risk = calculate_seismic_risk(test_params)

print("✅ Risk assessment function ready!")
print(f"\nTest risk assessment:")
print(f"{test_risk['color']} Risk Level: {test_risk['risk_level']} (Score: {test_risk['risk_score']}/100)")
for factor in test_risk['risk_factors']:
    print(f"   - {factor}")

✅ Risk assessment function ready!

Test risk assessment:
🔴 Risk Level: CRITICAL (Score: 75/100)
   - Moderate seismic zone (Zone 2, Z=0.2)
   - Poor soil condition (Class SD, S=1.35)
   - Old construction (built 1995, before 2000)


In [16]:
# ============================================================================
# SECTION 8: BNBC DESIGN PARAMETER CALCULATIONS
# ============================================================================

def calculate_design_parameters(params):
    """
    Calculate BNBC 2020 design parameters
    Based on Section 2.5.4.3 - Design Response Spectrum
    """
    results = {}

    # 1. Get Seismic Zone Coefficient (Z)
    if params['location']:
        zone_data = SEISMIC_ZONES.get(params['location'])
        if zone_data:
            results['Z'] = zone_data['Z']
            results['seismic_zone'] = zone_data['zone']
            results['intensity'] = zone_data['intensity']

    # 2. Get Soil Factor (S) and other soil parameters
    if params['soil_type']:
        soil_data = SOIL_CLASSES.get(params['soil_type'])
        if soil_data:
            results['S'] = soil_data['S']
            results['site_class'] = soil_data['class']
            results['TB'] = soil_data['TB']
            results['TC'] = soil_data['TC']
            results['TD'] = soil_data['TD']

    # 3. Get Importance Factor (I)
    if params['building_type']:
        importance_data = BUILDING_IMPORTANCE.get(params['building_type'])
        if importance_data:
            results['I'] = importance_data['I']
            results['occupancy_category'] = importance_data['category']
    else:
        results['I'] = 1.0  # Default
        results['occupancy_category'] = 'II'

    # 4. Calculate Peak Ground Acceleration (PGA)
    # PGA = (2/3) * Z * S (for design basis earthquake)
    if 'Z' in results and 'S' in results:
        results['PGA'] = round((2/3) * results['Z'] * results['S'], 3)

    # 5. Calculate Design Spectral Acceleration (Sa)
    # Simplified: Sa = (2/3) * Z * I * Cs / R
    # Where Cs = 2.5 * S * η (for period in constant acceleration range)
    # Assume R = 5 for typical RC structure, η = 1.0 for 5% damping
    if 'Z' in results and 'S' in results and 'I' in results:
        R = 5  # Typical response reduction factor
        eta = 1.0  # Damping correction factor (5% damping)
        Cs = 2.5 * results['S'] * eta  # Normalized spectrum coefficient

        results['Cs'] = round(Cs, 3)
        results['Sa'] = round((2/3) * results['Z'] * results['I'] * Cs / R, 4)
        results['R_assumed'] = R

    # 6. Determine Seismic Design Category
    if 'site_class' in results and 'seismic_zone' in results:
        zone = results['seismic_zone']
        site = results['site_class']
        category = results['occupancy_category']

        # Based on Table 6.2.18
        if category in ['I', 'II', 'III']:
            if zone == 1:
                sdc = 'B'
            elif zone == 2:
                sdc = 'C' if site in ['SA', 'SB', 'SC'] else 'D'
            elif zone == 3:
                sdc = 'C' if site == 'SA' else 'D'
            else:  # zone == 4
                sdc = 'D'
        else:  # Category IV
            if zone == 1:
                sdc = 'C'
            else:
                sdc = 'D'

        results['seismic_design_category'] = sdc

    return results

# Test design parameter calculation
test_params = {
    'location': 'Dhaka',
    'soil_type': 'marshland',
    'building_type': 'residential',
    'construction_year': 1995,
    'height': None
}

test_design = calculate_design_parameters(test_params)

print("✅ Design parameter calculation function ready!")
print("\nTest design parameters:")
for key, value in test_design.items():
    print(f"   {key}: {value}")

✅ Design parameter calculation function ready!

Test design parameters:
   Z: 0.2
   seismic_zone: 2
   intensity: Moderate
   S: 1.35
   site_class: SD
   TB: 0.2
   TC: 0.8
   TD: 2.0
   I: 1.0
   occupancy_category: II
   PGA: 0.18
   Cs: 3.375
   Sa: 0.09
   R_assumed: 5
   seismic_design_category: D


In [17]:
# ============================================================================
# SECTION 9: RAG QUERY FUNCTION
# ============================================================================

def query_bnbc_code(question, vectorstore, k=5):
    """
    Query BNBC code using RAG
    Returns relevant code sections
    """
    print("\n📚 Retrieving relevant BNBC code sections...")
    docs = vectorstore.similarity_search(question, k=k)

    print(f"   ✓ Found {len(docs)} relevant sections\n")

    print("=" * 80)
    print("RELEVANT BNBC CODE SECTIONS")
    print("=" * 80)

    for i, doc in enumerate(docs, 1):
        page = doc.metadata.get('page', 'N/A')
        print(f"\n[Section {i} - Page {page}]")
        print("-" * 80)
        print(doc.page_content)
        print()

    return docs

# Test RAG query
test_question = "What are the seismic zone coefficients for different cities?"
test_docs = query_bnbc_code(test_question, vectorstore, k=3)

print(f"✅ RAG query function ready!")


📚 Retrieving relevant BNBC code sections...
   ✓ Found 3 relevant sections

RELEVANT BNBC CODE SECTIONS

[Section 1 - Page 0]
--------------------------------------------------------------------------------
earthquake ground motions in order to minimize the risk to life for all 
structures, to increase the expected performance of higher occupancy structures 
as compared to ordinary structures, and to improve the capability of essential 
structures to function after an earthquake.  It is not economically feasible to 
design and construct buildings without any damage for a major earthquake 
event. The intent is therefore to allow inelastic deformation and structural 
damage at preferred locations in the structure without endangering structural 
integrity and to prevent structural collapse during a major earthquake. 
The seismic zoning map (Fig. 6.2.24) divides the country into four seismic zones 
with different expected levels of intensity of ground motion. Each seismic zone 
has a zone

In [18]:
# ============================================================================
# SECTION 10: GENERATE RECOMMENDATIONS
# ============================================================================

def generate_recommendations(risk, design_params, params):
    """Generate engineering recommendations based on risk and parameters"""
    recommendations = []

    # 1. Risk-based recommendations
    if risk['risk_level'] == 'CRITICAL':
        recommendations.append(
            "🚨 CRITICAL RISK LEVEL - IMMEDIATE ACTION REQUIRED:\n"
            "   • Evacuate and restrict building access immediately\n"
            "   • Engage licensed structural engineer within 24 hours\n"
            "   • Conduct detailed structural assessment (RAJUK approved)\n"
            "   • Seismic retrofitting URGENTLY needed\n"
            "   • Do not allow occupancy until certified safe"
        )
    elif risk['risk_level'] == 'HIGH':
        recommendations.append(
            "⚠️  HIGH RISK LEVEL - PRIORITY ACTION NEEDED:\n"
            "   • Schedule detailed structural evaluation within 1 week\n"
            "   • Seismic retrofitting strongly recommended\n"
            "   • Plan for strengthening within 3-6 months\n"
            "   • Limit occupancy to essential use only\n"
            "   • Monitor regularly for new cracks/damage"
        )
    elif risk['risk_level'] == 'MEDIUM':
        recommendations.append(
            "⚡ MEDIUM RISK LEVEL - MONITORING REQUIRED:\n"
            "   • Periodic structural inspection recommended (yearly)\n"
            "   • Consider minor strengthening measures\n"
            "   • Monitor for cracks, settlements, and damage\n"
            "   • Plan for retrofitting within 1-2 years\n"
            "   • Keep structural drawings updated"
        )
    else:
        recommendations.append(
            "✅ LOW RISK LEVEL - ROUTINE MAINTENANCE:\n"
            "   • Continue routine maintenance and inspection\n"
            "   • Visual inspection every 6 months sufficient\n"
            "   • Follow general building code compliance\n"
            "   • No immediate retrofitting required"
        )

    # 2. Design parameter based recommendations
    if 'Z' in design_params:
        Z = design_params['Z']
        zone = design_params.get('seismic_zone', 'N/A')
        PGA = design_params.get('PGA', 'N/A')

        recommendations.append(
            f"📊 BNBC 2020 DESIGN REQUIREMENTS:\n"
            f"   • Seismic Zone: Zone {zone} ({design_params.get('intensity', 'N/A')})\n"
            f"   • Zone Coefficient (Z): {Z}\n"
            f"   • Peak Ground Acceleration (PGA): {PGA}g\n"
            f"   • Soil Factor (S): {design_params.get('S', 'N/A')}\n"
            f"   • Site Class: {design_params.get('site_class', 'N/A')}\n"
            f"   • Importance Factor (I): {design_params.get('I', 'N/A')}\n"
            f"   • Seismic Design Category: {design_params.get('seismic_design_category', 'N/A')}"
        )

    # 3. Soil-specific recommendations
    if params['soil_type']:
        soil_data = SOIL_CLASSES.get(params['soil_type'])
        if soil_data and soil_data['class'] in ['SD', 'SE']:
            recommendations.append(
                "🏗️  FOUNDATION REQUIREMENTS (Poor Soil):\n"
                "   • Deep foundation (piles) MANDATORY\n"
                "   • Pile capacity: Conduct load test\n"
                "   • Soil improvement may be required\n"
                "   • DO NOT use shallow foundations\n"
                "   • Check for liquefaction potential (BNBC Sec 2.5.3.1)\n"
                "   • Conduct detailed geotechnical investigation\n"
                "   • Consider mat foundation as alternative"
            )
        elif soil_data and soil_data['class'] == 'SC':
            recommendations.append(
                "🏗️  FOUNDATION REQUIREMENTS (Medium Soil):\n"
                "   • Deep foundation recommended for buildings > 3 stories\n"
                "   • Shallow foundation possible with proper design\n"
                "   • Conduct standard penetration test (SPT)\n"
                "   • Follow BNBC Chapter 3 for foundation design"
            )

    # 4. Construction year based recommendations
    if params['construction_year']:
        year = params['construction_year']

        if year < 1993:
            recommendations.append(
                "🔧 RETROFITTING PRIORITY (Pre-BNBC 1993):\n"
                "   • Building designed before earthquake code\n"
                "   • Lacks adequate seismic detailing\n"
                "   • Column jacketing ESSENTIAL (3-4 inch thick)\n"
                "   • Beam-column joint strengthening required\n"
                "   • Add RC shear walls at strategic locations\n"
                "   • FRP wrapping for critical columns\n"
                "   • Foundation underpinning if settlement detected\n"
                "   • Estimated cost: ৳800-1200 per sqft"
            )
        elif year < 2020:
            recommendations.append(
                "🔧 RETROFITTING RECOMMENDED (Pre-BNBC 2020):\n"
                "   • Update to BNBC 2020 standards\n"
                "   • Selective strengthening of weak elements\n"
                "   • Check beam-column joint adequacy\n"
                "   • Consider adding shear walls if space permits\n"
                "   • Estimated cost: ৳400-600 per sqft"
            )

    # 5. Building type specific recommendations
    if params['building_type'] in ['hospital', 'emergency_shelter', 'fire_station', 'power_plant']:
        recommendations.append(
            "🏥 ESSENTIAL FACILITY REQUIREMENTS (I=1.5):\n"
            "   • Higher design standards apply (Occupancy Category IV)\n"
            "   • Must remain operational after earthquake\n"
            "   • Seismic Design Category D likely applicable\n"
            "   • Use Special RC moment frames or shear walls\n"
            "   • Redundant structural systems required\n"
            "   • Backup power and emergency systems mandatory\n"
            "   • Annual structural inspection required"
        )

    # 6. Height-based recommendations
    if params['height']:
        height = params['height']

        if height > 40:
            recommendations.append(
                "🏢 HIGH-RISE BUILDING REQUIREMENTS:\n"
                "   • Dynamic analysis (Response Spectrum) MANDATORY\n"
                "   • Wind tunnel test may be required\n"
                "   • P-Delta effects must be considered\n"
                "   • Detailed foundation design required\n"
                "   • RAJUK approval with structural certificate\n"
                "   • Special inspection during construction"
            )
        elif height > 12:
            recommendations.append(
                "🏘️  MID-RISE BUILDING REQUIREMENTS:\n"
                "   • Equivalent static analysis acceptable\n"
                "   • Response spectrum analysis recommended\n"
                "   • Proper drift control essential\n"
                "   • Strong column-weak beam design principle"
            )

    return recommendations

# Test recommendations
test_recommendations = generate_recommendations(test_risk, test_design, test_params)

print("✅ Recommendation generator ready!")
print("\nTest recommendations:")
for i, rec in enumerate(test_recommendations, 1):
    print(f"\n{rec}")

✅ Recommendation generator ready!

Test recommendations:

🚨 CRITICAL RISK LEVEL - IMMEDIATE ACTION REQUIRED:
   • Evacuate and restrict building access immediately
   • Engage licensed structural engineer within 24 hours
   • Conduct detailed structural assessment (RAJUK approved)
   • Seismic retrofitting URGENTLY needed
   • Do not allow occupancy until certified safe

📊 BNBC 2020 DESIGN REQUIREMENTS:
   • Seismic Zone: Zone 2 (Moderate)
   • Zone Coefficient (Z): 0.2
   • Peak Ground Acceleration (PGA): 0.18g
   • Soil Factor (S): 1.35
   • Site Class: SD
   • Importance Factor (I): 1.0
   • Seismic Design Category: D

🏗️  FOUNDATION REQUIREMENTS (Poor Soil):
   • Deep foundation (piles) MANDATORY
   • Pile capacity: Conduct load test
   • Soil improvement may be required
   • DO NOT use shallow foundations
   • Check for liquefaction potential (BNBC Sec 2.5.3.1)
   • Conduct detailed geotechnical investigation
   • Consider mat foundation as alternative

🔧 RETROFITTING RECOMMENDED 

In [19]:
# ============================================================================
# SECTION 11: COMPLETE QUERY FUNCTION
# ============================================================================

def complete_bnbc_query(question, vectorstore):
    """
    Complete BNBC query with all features:
    1. Parameter extraction
    2. Risk assessment
    3. Design calculations
    4. Code retrieval
    5. Recommendations
    """

    print("\n" + "=" * 80)
    print("COMPLETE BNBC 2020 ENGINEERING ANALYSIS")
    print("=" * 80)
    print(f"\n📝 Query: {question}\n")

    # Step 1: Extract Parameters
    print("🔍 STEP 1: Extracting Engineering Parameters")
    print("-" * 80)
    params = extract_all_parameters(question)

    for key, value in params.items():
        if value:
            print(f"   ✓ {key}: {value}")
        else:
            print(f"   ✗ {key}: Not specified")

    # Step 2: Risk Assessment
    print("\n⚠️  STEP 2: Seismic Risk Assessment")
    print("-" * 80)
    risk = calculate_seismic_risk(params)

    print(f"\n   {risk['color']} RISK LEVEL: {risk['risk_level']}")
    print(f"   📊 Risk Score: {risk['risk_score']}/100\n")
    print("   Risk Factors:")
    for factor in risk['risk_factors']:
        print(f"   • {factor}")

    # Step 3: Design Parameters
    print("\n📐 STEP 3: BNBC Design Parameter Calculations")
    print("-" * 80)
    design_params = calculate_design_parameters(params)

    if design_params:
        print("\n   Design Parameters (BNBC 2020):")
        for key, value in design_params.items():
            print(f"   • {key}: {value}")
    else:
        print("   ⚠️  Insufficient data for design calculations")

    # Step 4: Retrieve Code Sections
    print("\n📚 STEP 4: Retrieving Relevant BNBC Code Sections")
    print("-" * 80)
    docs = vectorstore.similarity_search(question, k=5)

    print(f"\n   Found {len(docs)} relevant sections:")
    for i, doc in enumerate(docs, 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:100].replace('\n', ' ')
        print(f"   [{i}] Page {page}: {preview}...")

    # Step 5: Generate Recommendations
    print("\n💡 STEP 5: Engineering Recommendations")
    print("-" * 80)
    recommendations = generate_recommendations(risk, design_params, params)

    for rec in recommendations:
        print(f"\n{rec}")

    # Summary
    print("\n" + "=" * 80)
    print("ANALYSIS SUMMARY")
    print("=" * 80)
    print(f"\nLocation: {params.get('location', 'Not specified')}")
    print(f"Building Type: {params.get('building_type', 'Not specified')}")
    print(f"Soil Type: {params.get('soil_type', 'Not specified')}")
    print(f"Construction Year: {params.get('construction_year', 'Not specified')}")
    print(f"\nRisk Level: {risk['color']} {risk['risk_level']} ({risk['risk_score']}/100)")
    print(f"Seismic Zone: Zone {design_params.get('seismic_zone', 'N/A')} (Z={design_params.get('Z', 'N/A')})")
    print(f"PGA: {design_params.get('PGA', 'N/A')}g")
    print(f"Seismic Design Category: {design_params.get('seismic_design_category', 'N/A')}")
    print("=" * 80 + "\n")

    return {
        'parameters': params,
        'risk_assessment': risk,
        'design_parameters': design_params,
        'code_sections': docs,
        'recommendations': recommendations
    }

print("✅ Complete query function ready!")

✅ Complete query function ready!


In [20]:
# ============================================================================
# SECTION 12: VALIDATION AGAINST MANUAL CALCULATIONS
# ============================================================================

def validate_calculations(manual_values, calculated_values):
    """
    Validate AI calculations against manual engineering calculations
    Tolerance: 5% error acceptable
    """

    print("\n" + "=" * 80)
    print("VALIDATION: AI vs MANUAL CALCULATIONS")
    print("=" * 80)

    validation_results = []
    tolerance = 0.05  # 5% tolerance

    all_pass = True

    for param, manual_val in manual_values.items():
        if param in calculated_values:
            calc_val = calculated_values[param]

            if isinstance(manual_val, (int, float)) and isinstance(calc_val, (int, float)):
                # Calculate error
                if manual_val != 0:
                    error = abs(calc_val - manual_val) / manual_val
                else:
                    error = 0 if calc_val == 0 else 1

                # Determine status
                if error <= tolerance:
                    status = "✅ PASS"
                else:
                    status = "❌ FAIL"
                    all_pass = False

                validation_results.append({
                    'parameter': param,
                    'manual': manual_val,
                    'calculated': calc_val,
                    'error_percent': error * 100,
                    'status': status
                })

                print(f"\n{param}:")
                print(f"   Manual Value:      {manual_val}")
                print(f"   Calculated Value:  {calc_val}")
                print(f"   Error:             {error*100:.2f}%")
                print(f"   Tolerance:         {tolerance*100:.0f}%")
                print(f"   Status:            {status}")
            else:
                print(f"\n{param}:")
                print(f"   Manual:       {manual_val}")
                print(f"   Calculated:   {calc_val}")
                print(f"   Status:       ℹ️  Non-numeric comparison")
        else:
            print(f"\n{param}:")
            print(f"   Status:       ⚠️  Not calculated by AI")

    # Overall result
    print("\n" + "=" * 80)
    if all_pass:
        print("✅ VALIDATION PASSED - All parameters within tolerance")
    else:
        print("❌ VALIDATION FAILED - Some parameters exceed tolerance")
    print("=" * 80 + "\n")

    return validation_results

print("✅ Validation function ready!")

✅ Validation function ready!


In [21]:
# ============================================================================
# SECTION 13: EXAMPLE USAGE
# ============================================================================

print("\n" + "#" * 80)
print("# EXAMPLE 1: DHAKA RESIDENTIAL BUILDING")
print("#" * 80)

query1 = """
I have a residential building in Dhaka built in 1995 on marshland soil.
The building is 5 stories tall. What is the seismic risk and what should I do?
"""

result1 = complete_bnbc_query(query1, vectorstore)

# ============================================================================

print("\n" + "#" * 80)
print("# EXAMPLE 2: SYLHET HOSPITAL")
print("#" * 80)

query2 = """
We are planning to build a hospital in Sylhet on dense sand soil.
What are the BNBC 2020 seismic design requirements?
"""

result2 = complete_bnbc_query(query2, vectorstore)

# ============================================================================

print("\n" + "#" * 80)
print("# EXAMPLE 3: VALIDATION TEST")
print("#" * 80)

# Manual calculations for Dhaka residential on marshland
manual_calculations = {
    'Z': 0.20,           # Zone 2 coefficient
    'S': 1.35,           # Site class SD
    'I': 1.00,           # Occupancy category II
    'PGA': 0.180,        # (2/3) * 0.20 * 1.35 = 0.18
    'seismic_zone': 2,
}

calculated = result1['design_parameters']

validation_results = validate_calculations(manual_calculations, calculated)

print("✅ All examples completed!")


################################################################################
# EXAMPLE 1: DHAKA RESIDENTIAL BUILDING
################################################################################

COMPLETE BNBC 2020 ENGINEERING ANALYSIS

📝 Query: 
I have a residential building in Dhaka built in 1995 on marshland soil. 
The building is 5 stories tall. What is the seismic risk and what should I do?


🔍 STEP 1: Extracting Engineering Parameters
--------------------------------------------------------------------------------
   ✓ location: Dhaka
   ✓ soil_type: marshland
   ✓ building_type: residential
   ✓ construction_year: 1995
   ✗ height: Not specified

⚠️  STEP 2: Seismic Risk Assessment
--------------------------------------------------------------------------------

   🔴 RISK LEVEL: CRITICAL
   📊 Risk Score: 75/100

   Risk Factors:
   • Moderate seismic zone (Zone 2, Z=0.2)
   • Poor soil condition (Class SD, S=1.35)
   • Old construction (built 1995, before 2000)

📐 STEP 3

In [23]:
query1 = """
I have a residential building in Dhaka built in 1995 on marshland soil.
The building is 5 stories tall. What is the seismic risk and what should I do?
"""

result1 = complete_bnbc_query(query1, vectorstore)


COMPLETE BNBC 2020 ENGINEERING ANALYSIS

📝 Query: 
I have a residential building in Dhaka built in 1995 on marshland soil. 
The building is 5 stories tall. What is the seismic risk and what should I do?


🔍 STEP 1: Extracting Engineering Parameters
--------------------------------------------------------------------------------
   ✓ location: Dhaka
   ✓ soil_type: marshland
   ✓ building_type: residential
   ✓ construction_year: 1995
   ✗ height: Not specified

⚠️  STEP 2: Seismic Risk Assessment
--------------------------------------------------------------------------------

   🔴 RISK LEVEL: CRITICAL
   📊 Risk Score: 75/100

   Risk Factors:
   • Moderate seismic zone (Zone 2, Z=0.2)
   • Poor soil condition (Class SD, S=1.35)
   • Old construction (built 1995, before 2000)

📐 STEP 3: BNBC Design Parameter Calculations
--------------------------------------------------------------------------------

   Design Parameters (BNBC 2020):
   • Z: 0.2
   • seismic_zone: 2
   • intensity: M

In [26]:
query1 = """ I want to make a residential building at Old dhaka.
The building is 5 stories tall almost 60 feet tall. What is the seismic risk and what should I do?
"""
result1 = complete_bnbc_query(query1, vectorstore)




COMPLETE BNBC 2020 ENGINEERING ANALYSIS

📝 Query:  I want to make a residential building at Old dhaka.
The building is 5 stories tall almost 60 feet tall. What is the seismic risk and what should I do?


🔍 STEP 1: Extracting Engineering Parameters
--------------------------------------------------------------------------------
   ✓ location: Dhaka
   ✗ soil_type: Not specified
   ✓ building_type: residential
   ✗ construction_year: Not specified
   ✗ height: Not specified

⚠️  STEP 2: Seismic Risk Assessment
--------------------------------------------------------------------------------

   🟢 RISK LEVEL: LOW
   📊 Risk Score: 20/100

   Risk Factors:
   • Moderate seismic zone (Zone 2, Z=0.2)

📐 STEP 3: BNBC Design Parameter Calculations
--------------------------------------------------------------------------------

   Design Parameters (BNBC 2020):
   • Z: 0.2
   • seismic_zone: 2
   • intensity: Moderate
   • I: 1.0
   • occupancy_category: II

📚 STEP 4: Retrieving Relevant BNBC C